In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

from scipy.stats import spearmanr
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import PredefinedSplit, GridSearchCV


from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
import pytorch_lightning as pl
import torch
import wandb

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_lo_metrics
from mlp_model import MoleculeDataset, LoModel

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/lo/drd2/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/lo/drd2/test_1.csv', index_col=0)

test

,smiles,value,cluster
0,Brc1ccc(-[n+]2cc[n+](Cc3ccccc3)cc2)c2cc[nH]c12,7.717691,11
1,Brc1cccc(N2CCN(Cc3cnn4ccccc34)CC2)n1,6.748370,26
2,C#CC1=CCC(N(CCC)CCCCn2cc(-c3ccc(-c4ccccc4)cc3)...,6.490481,14
3,C#CCN(CCN1CCN(c2ccccc2)CC1)C1CCc2ccc(O)cc2C1,6.609065,32
4,C1=C(c2ccccc2)CCN(Cc2cnn(-c3ccccc3)c2)C1,7.473269,12
...,...,...,...
262,c1ccc2c(c1)N=C(N1CCNCC1)c1ccccc1S2,7.420216,6
263,c1cnc(N2CCN(CCCOc3ccc(-c4nc5ccccc5[nH]4)cc3)CC...,6.568636,35
264,c1cnc(N2CCN(CCCOc3ccc(-c4nc5ccccc5o4)cc3)CC2)nc1,6.701147,35
265,c1cnc(N2CCN(Cc3c[nH]c4ncccc34)CC2)nc1,5.931443,12


In [4]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]
train_fps = np.array(train_fps, dtype=np.float32)

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]
test_fps = np.array(test_fps, dtype=np.float32)


In [5]:
train_dataset = MoleculeDataset(train_fps, np.array(train['value'], dtype=np.float32))
test_dataset = MoleculeDataset(test_fps, np.array(test['value'], dtype=np.float32))

In [6]:
params = {
    'layers': [1024, 32, 32],
    'use_dropout': False,
    'dropout': 0.0,
    'lr': 0.01,
    'l2': 0.01,
    'batch_size': 32,
}

In [7]:
param_dict = {
    'layers': [
        [1024, 32, 32],
        [1024, 16, 16],
        [1024, 32],
        [1024, 8, 4],
        [1024, 4]
    ],
    'use_dropout': [True],
    'dropout': [0.0, 0.0, 0.2, 0.4, 0.6],
    'lr': [0.01],
    'l2': [0.0, 0.0, 0.001, 0.005, 0.01],
    'batch_size': [32]
}

In [8]:
from sklearn.model_selection import ParameterSampler
from pytorch_lightning.callbacks import EarlyStopping

hyperparam_list = list(ParameterSampler(param_dict, n_iter=30))

best_score = 0.0
best_hyperparam = None

for i, hyperparam in enumerate(hyperparam_list):
    early_stop_callback = EarlyStopping(
        monitor='test_spearman',  # the metric to monitor
        min_delta=0.00,  # minimum change in the monitored quantity to qualify as an improvement
        patience=10,  # number of epochs with no improvement after which training will be stopped
        verbose=False,  # verbosity mode
        mode='max'  # 'min' for minimizing the metric, 'max' for maximizing
    )

    print('HYPERPARAMETER', i)
    print(hyperparam)

    train_dataloader = DataLoader(train_dataset, batch_size=hyperparam['batch_size'], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=hyperparam['batch_size'], shuffle=False)

    model = LoModel(hyperparam, train, test)
    wandb.init(project='drd2_lo_ecfp4')
    wandb_logger = WandbLogger(project='drd2_lo_ecfp4')
    trainer = pl.Trainer(max_epochs=150, logger=wandb_logger, accelerator="gpu", devices=[1], default_root_dir=None, callbacks=[early_stop_callback], num_sanity_val_steps=0)
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)
    score = early_stop_callback.best_score.detach().cpu().numpy()
    if score > best_score:
        best_score = score
        best_hyperparam = hyperparam

HYPERPARAMETER 0
{'use_dropout': True, 'lr': 0.01, 'layers': [1024, 32], 'l2': 0.001, 'dropout': 0.2, 'batch_size': 32}
Sequential(
  (0): Linear(in_features=1024, out_features=32, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=32, out_features=1, bias=True)
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: steshinss. Use `wandb login --relogin` to force relogin


/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 32.8 K
-------------------------------------
32.8 K    Trainable params
0         Non-trainable params
32.8 K    Total params
0.131     Total estimated model params size (MB)
wandb: WARNING Serializing object of type DataFrame that is 291558 bytes
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [9]:
best_hyperparam

{'use_dropout': True,
 'lr': 0.01,
 'layers': [1024, 32],
 'l2': 0.005,
 'dropout': 0.6,
 'batch_size': 32}

In [10]:
best_hyperparam = {'use_dropout': True,
    'lr': 0.01,
    'layers': [1024, 32],
    'l2': 0.005,
    'dropout': 0.6,
    'batch_size': 32
}

In [11]:
from pytorch_lightning.callbacks import EarlyStopping
early_stop_callback = EarlyStopping(
    monitor='test_spearman',  # the metric to monitor
    min_delta=0.00,  # minimum change in the monitored quantity to qualify as an improvement
    patience=10,  # number of epochs with no improvement after which training will be stopped
    verbose=False,  # verbosity mode
    mode='max'  # 'min' for minimizing the metric, 'max' for maximizing
)
train_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)

model = LoModel(best_hyperparam, train, test)
wandb.init(project='drd2_lo_ecfp4')
wandb_logger = WandbLogger(project='drd2_lo_ecfp4')
trainer = pl.Trainer(max_epochs=150, logger=wandb_logger, accelerator="gpu", devices=[1], default_root_dir=None, callbacks=[early_stop_callback], num_sanity_val_steps=0)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

Sequential(
  (0): Linear(in_features=1024, out_features=32, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.6, inplace=False)
  (3): Linear(in_features=32, out_features=1, bias=True)
)


epoch,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇██
test_mae,▄▆█▆▂▁▃▁▁▁▂▁▁
test_r2,▅▃▁▃▇█▆███▇██
test_spearman,▆██▇▇█▅▅▃▃▁▂▁
train_loss,█▄▃▃▂▃▂▃▂▁▃▁▁▂▂▂▁
trainer/global_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇██
val_loss,▄▆█▆▂▁▃▁▁▁▂▁▁
epoch,12
test_mae,0.82189
test_r2,-0.69077
test_spearman,0.11904


/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 32.8 K
-------------------------------------
32.8 K    Trainable params
0         Non-trainable params
32.8 K    Total params
0.131     Total estimated model params size (MB)
wandb: WARNING Serializing object of type DataFrame that is 291558 bytes
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [12]:
all_preds = []
all_true = []
with torch.no_grad():
    for _, batch in enumerate(test_dataloader):
        X, y = batch
        y_pred = model(X)
        all_preds.append(y_pred)
        all_true.append(y)

In [13]:
all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
all_true = np.squeeze(torch.cat(all_true).cpu().numpy())

In [14]:
get_lo_metrics(test, all_preds)

{'r2': -0.7852144344010662,
 'spearman': 0.0708442512160913,
 'mae': 0.8368538236536118}

# Final Prediction

In [15]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]
    train_fps = np.array(train_fps, dtype=np.float32)

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]
    test_fps = np.array(test_fps, dtype=np.float32)

    train_dataset = MoleculeDataset(train_fps, np.array(train['value'], dtype=np.float32))
    test_dataset = MoleculeDataset(test_fps, np.array(test['value'], dtype=np.float32))

    train_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)

    model = LoModel(best_hyperparam, train, test)
    trainer = pl.Trainer(max_epochs=11, accelerator="gpu", devices=[1], default_root_dir=None, num_sanity_val_steps=0)
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

    train_result = train.copy()
    train_predict_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)
    all_preds = []
    with torch.no_grad():
        for _, batch in enumerate(train_predict_dataloader):
            X, _ = batch
            y_pred = model(X)
            all_preds.append(y_pred)
    all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
    train_result['preds'] = all_preds

    test_result = test.copy()
    all_preds = []
    with torch.no_grad():
        for _, batch in enumerate(test_dataloader):
            X, _ = batch
            y_pred = model(X)
            all_preds.append(y_pred)
    all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
    test_result['preds'] = all_preds

    return train_result, test_result

In [16]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/lo/drd2/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/lo/drd2/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/lo/drd2/mlp_ecfp4/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/lo/drd2/mlp_ecfp4/test_{i}.csv')


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /data/steshin/gero_benchmark/notebooks/models/lo/drd2/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 32.8 K
-------------------------------------
32.8 K    Trainable params
0         Non-trainable params
32.8 K    Total params
0.131     Total estimated model params size (MB)


Sequential(
  (0): Linear(in_features=1024, out_features=32, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.6, inplace=False)
  (3): Linear(in_features=32, out_features=1, bias=True)
)


/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=11` reached.
